In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive',force_remount=True)
os.chdir('/content/drive/My Drive/Brain-Tumor')

Mounted at /content/drive


**LBPH**

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def LBPH(grayImage):
    if grayImage is None:
        raise ValueError("Failed to load the image.")

    rows, columns = grayImage.shape
    localBinaryPatternImage = np.zeros_like(grayImage)

    for row in range(1, rows - 1):
        for col in range(1, columns - 1):
            centerPixel = grayImage[row, col]
            pixel7 = grayImage[row - 1, col - 1] > centerPixel
            pixel6 = grayImage[row - 1, col] > centerPixel
            pixel5 = grayImage[row - 1, col + 1] > centerPixel
            pixel4 = grayImage[row, col + 1] > centerPixel
            pixel3 = grayImage[row + 1, col + 1] > centerPixel
            pixel2 = grayImage[row + 1, col] > centerPixel
            pixel1 = grayImage[row + 1, col - 1] > centerPixel
            pixel0 = grayImage[row, col - 1] > centerPixel
            localBinaryPatternImage[row, col] = np.uint8(
                pixel7 * 2 ** 7 + pixel6 * 2 ** 6 +
                pixel5 * 2 ** 5 + pixel4 * 2 ** 4 +
                pixel3 * 2 ** 3 + pixel2 * 2 ** 2 +
                pixel1 * 2 + pixel0
            )

    return localBinaryPatternImage



**GLCM_Features**

In [ ]:
import numpy as np
from skimage.feature import greycomatrix, greycoprops
      skimage.feature.graycomatrixLCM_Features1(image, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], properties=['energy', 'homogeneity']):
    # Compute the gray-level co-occurrence matrix
    glcm = greycomatrix(image, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)

    # Compute the desired GLCM properties
    features = []
    for prop in properties:
        prop_value = greycoprops(glcm, prop)
        features.extend(prop_value.flatten())

    return features

    out = {}
    out['autoc'] = np.zeros(size_glcm_3)
    out['contr'] = np.zeros(size_glcm_3)
    out['corrm'] = np.zeros(size_glcm_3)
    out['corrp'] = np.zeros(size_glcm_3)
    out['cprom'] = np.zeros(size_glcm_3)
    out['cshad'] = np.zeros(size_glcm_3)
    out['dissi'] = np.zeros(size_glcm_3)
    out['energ'] = np.zeros(size_glcm_3)
    out['entro'] = np.zeros(size_glcm_3)
    out['homom'] = np.zeros(size_glcm_3)
    out['homop'] = np.zeros(size_glcm_3)
    out['maxpr'] = np.zeros(size_glcm_3)
    out['sosvh'] = np.zeros(size_glcm_3)
    out['savgh'] = np.zeros(size_glcm_3)
    out['svarh'] = np.zeros(size_glcm_3)
    out['senth'] = np.zeros(size_glcm_3)
    out['dvarh'] = np.zeros(size_glcm_3)
    out['denth'] = np.zeros(size_glcm_3)
    out['inf1h'] = np.zeros(size_glcm_3)
    out['inf2h'] = np.zeros(size_glcm_3)
    out['indnc'] = np.zeros(size_glcm_3)
    out['idmnc'] = np.zeros(size_glcm_3)

    if pairs == 1:
        newn = 1
        glcm = np.zeros((size_glcm_1, size_glcm_2, size_glcm_3//2))
        for nglcm in range(0, size_glcm_3, 2):
            glcm[:, :, newn-1] = glcmin[:, :, nglcm] + glcmin[:, :, nglcm+1]
            newn += 1
    elif pairs == 0:
        glcm = glcmin

    glcm_sum = np.zeros(size_glcm_3)
    glcm_mean = np.zeros(size_glcm_3)
    glcm_var = np.zeros(size_glcm_3)
    u_x = np.zeros(size_glcm_3)
    u_y = np.zeros(size_glcm_3)
    s_x = np.zeros(size_glcm_3)
    s_y = np.zeros(size_glcm_3)
    p_x = np.zeros((size_glcm_1, size_glcm_3))
    p_y = np.zeros((size_glcm_2, size_glcm_3))

    for k in range(size_glcm_3):
        glcm_sum[k] = np.sum(glcm[:, :, k])
        glcm_mean[k] = np.mean(glcm[:, :, k])
        glcm_var[k] = np.var(glcm[:, :, k])

        for i in range(size_glcm_1):
            for j in range(size_glcm_2):
                p_x[i, k] += glcm[i, j, k]
                p_y[j, k] += glcm[i, j, k]

                u_x[k] += i * glcm[i, j, k]
                u_y[k] += j * glcm[i, j, k]

        s_x[k] = np.sqrt(np.var(u_x[k]))
        s_y[k] = np.sqrt(np.var(u_y[k]))

        u_x[k] /= glcm_sum[k]
        u_y[k] /= glcm_sum[k]

        tmp_sum = np.sum(glcm[:, :, k])

        p_x[:, k] /= tmp_sum
        p_y[:, k] /= tmp_sum

        p_xminusy = np.zeros((size_glcm_1, size_glcm_2, size_glcm_3))
        p_xplusy = np.zeros((2*size_glcm_1-1, size_glcm_3))

        for i in range(size_glcm_1):
            for j in range(size_glcm_2):
                p_xminusy[i, j, k] = glcm[i, j, k] / tmp_sum
                p_xplusy[i+j, k] += glcm[i, j, k] / tmp_sum

        p_xminusy_sum = np.sum(p_xminusy[:, :, k])
        p_xplusy_sum = np.sum(p_xplusy[:, k])

        autoc = 0.0
        contrast = 0.0
        corrm = 0.0
        corrp = 0.0
        cprom = 0.0
        cshad = 0.0
        dissimilarity = 0.0
        energy = 0.0
        entropy = 0.0
        homom = 0.0
        homop = 0.0
        maxpr = 0.0
        sosvh = 0.0
        savgh = 0.0
        svarh = 0.0
        senth = 0.0
        dvarh = 0.0
        denth = 0.0
        inf1h = 0.0
        inf2h = 0.0
        indnc = 0.0
        idmnc = 0.0

        for i in range(size_glcm_1):
            for j in range(size_glcm_2):
                autoc += glcm[i, j, k] * (i == j)
                contrast += (i - j) ** 2 * glcm[i, j, k]
                corrm += i * j * glcm[i, j, k]
                cprom += ((i + j - u_x[k] - u_y[k]) ** 4) * glcm[i, j, k]
                cshad += ((i + j - u_x[k] - u_y[k]) ** 3) * glcm[i, j, k]
                dissimilarity += np.abs(i - j) * glcm[i, j, k]
                energy += glcm[i, j, k] ** 2
                entropy += -p_xminusy[i, j, k] * np.log(p_xminusy[i, j, k] + 1e-10)
                homom += glcm[i, j, k] / (1.0 + np.abs(i - j))
                homop += glcm[i, j, k] / (1.0 + (i - j) ** 2)
                maxpr = max(maxpr, glcm[i, j, k])
                sosvh += glcm[i, j, k] * ((i - u_x[k]) ** 2)
                savgh += glcm[i, j, k] * (i + j)
                svarh += glcm[i, j, k] * ((i - u_x[k]) ** 2)
                senth += glcm[i, j, k] * np.log(glcm[i, j, k] + 1e-10)
                dvarh += glcm[i, j, k] * ((i - u_x[k]) ** 2) * ((j - u_y[k]) ** 2)
                denth += glcm[i, j, k] * np.log(p_xminusy[i, j, k] + 1e-10)
                inf1h += (i - u_x[k]) ** 2 * np.log(p_x[i, k] + 1e-10)
                inf2h += (i - u_x[k]) ** 2 * np.log(p_y[j, k] + 1e-10)
                indnc += glcm[i, j, k] / (1.0 + np.abs(i - j))
                idmnc += glcm[i, j, k] / (1.0 + (i - j) ** 2)

        out['autoc'][k] = autoc
        out['contrast'][k] = contrast
        out['corrm'][k] = corrm
        out['corrp'][k] = corrp
        out['cprom'][k] = cprom
        out['cshad'][k] = cshad
        out['dissi'][k] = dissimilarity
        out['energ'][k] = energy
        out['entro'][k] = entropy
        out['homom'][k] = homom
        out['homop'][k] = homop
        out['maxpr'][k] = maxpr
        out['sosvh'][k] = sosvh
        out['savgh'][k] = savgh
        out['svarh'][k] = svarh
        out['senth'][k] = senth
        out['dvarh'][k] = dvarh
        out['denth'][k] = denth
        out['inf1h'][k] = inf1h
        out['inf2h'][k] = inf2h
        out['indnc'][k] = indnc
        out['idmnc'][k] = idmnc

    return out




**Stats_features**

In [ ]:
import numpy as np
from skimage.io import imread
from skimage.measure import label, regionprops
from scipy.stats import kurtosis, skew
from skimage.morphology import convex_hull_image
from skimage.measure import perimeter as skimage_perimeter

def stats_features(img_path):
    # Read the image
    img = imread(img_path, as_gray=True)

    # Original image
    area = np.sum(img)  # Area
    BW2 = np.pad(img, pad_width=1, mode='constant')
    img_perimeter = np.sum(skimage_perimeter(BW2))  # Perimeter
    RMS = np.sqrt(np.mean(img**2))  # RMS
    a = np.sum(img.astype(float))
    smoothness = 1 - (1 / (1 + a))  # Smoothness
    kurt = kurtosis(img.flatten())  # Kurtosis
    skewness = skew(img.flatten())  # Skewness
    compac = (img_perimeter**2) / (4 * np.pi * area)  # Compactness

    # Convex hull image
    H = convex_hull_image(img)
    CI = np.pad(H, pad_width=1, mode='constant')
    cpiremeter = np.sum(skimage_perimeter(CI))
    Carea = np.sum(H)
    convexity = cpiremeter / img_perimeter

    # Eccentricity
    labeled_img = label(img)
    props = regionprops(labeled_img)
    centroid = props[0].centroid
    major_axis_length = props[0].major_axis_length
    minor_axis_length = props[0].minor_axis_length
    Min = minor_axis_length
    Maj = major_axis_length
    eccentricity = Min / Maj
    circularity = img_perimeter**2 / (4 * np.pi * area)
    ECD = np.sqrt(4 * area / np.pi)  # Equivalent Circular Diameter
    effective_diameter = 2 * np.sqrt(area / np.pi)

    out = [area, img_perimeter, RMS, smoothness, kurt, skewness, centroid, major_axis_length, minor_axis_length,
           compac, eccentricity, Carea, convexity, circularity, cpiremeter, ECD, effective_diameter]

    return out




**HaralickTextureFeatures**

In [ ]:
import numpy as np

def haralickTextureFeatures(coOcMat, xFeatures=None):
    """
    Calculates all Haralick Features.

    Args:
        coOcMat (ndarray): Co-Occurrence Matrix, a 2D numpy array [nxm]
        xFeatures (list): List of feature(s) to calculate (optional)

    Returns:
        ndarray: Vector with the calculated feature(s)

    """

    # Check if xFeatures is None, set it to calculate all features
    if xFeatures is None:
        xFeatures = list(range(1, 15))

    # Check coOcMat for dimensions
    if coOcMat.ndim != 2:
        raise ValueError("coOcMat must be a two-dimensional matrix.")

    # Initialize x
    x = np.zeros(14)

    # Normalize coOcMat
    coOcMat = coOcMat / np.sum(coOcMat)

    # Some pre-calculation
    if any([f in [2, 3, 4, 5, 6, 7, 8, 10, 11, 14] for f in xFeatures]):
        sizecoOcMat = coOcMat.shape
        col, row = np.meshgrid(np.arange(1, sizecoOcMat[1] + 1), np.arange(1, sizecoOcMat[0] + 1))

    if any([f in [3, 10] for f in xFeatures]):
        rowMean = np.sum(row * coOcMat)
        colMean = np.sum(col * coOcMat)
        rowStd = np.sqrt(np.sum((row - rowMean) ** 2 * coOcMat))
        colStd = np.sqrt(np.sum((col - colMean) ** 2 * coOcMat))

    if any([f in [12, 13, 14] for f in xFeatures]):
        rowCoOcMat = np.sum(coOcMat, axis=1)
        colCoOcMat = np.sum(coOcMat, axis=0)

    if any([f in [6, 7, 8] for f in xFeatures]):
        start = -(sizecoOcMat[0] - 1)
        stop = sizecoOcMat[0] - 1
        coOcMat90 = np.rot90(coOcMat)
        p_XplusY = np.zeros((2 * sizecoOcMat[0]) - 1)
        k = 0
        for index in range(start, stop + 1):
            p_XplusY[k] = np.sum(np.diag(coOcMat90, index))
            k += 1

    if any([f in [10, 11] for f in xFeatures]):
        start = 1
        stop = sizecoOcMat[0] - 1
        p_XminusY = np.zeros(sizecoOcMat[0])
        p_XminusY[0] = np.sum(np.diag(coOcMat, 0))
        k = 1
        for index in range(start, stop + 1):
            p_XminusY[k] = np.sum(np.concatenate((np.diag(coOcMat, index), np.diag(coOcMat, -index))))
            k += 1

    # Haralick Feature Calculations
    for f in xFeatures:
        if f == 1:  # Energy (Angular Second Moment)
            x[0]


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import local_binary_pattern
from scipy.stats import describe
from skimage.measure import regionprops

dataset_path = '/content/drive/My Drive/Brain-Tumor/Segemented_Images'
data = []

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        image_path = os.path.join(root, file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        data.append(image)
  

all_features = []


for i in range(len(data)):
    image = data[i].astype(np.float32) / 255.0
    
    # Statistical features
    stats_features = describe(image.ravel())
    
    # Haralick texture features
    texture_features = []
    glcm = greycomatrix(image.astype(np.uint8), [1], [0], levels=256)
    haralick_features = greycoprops(glcm, prop='homogeneity')
    texture_features.extend(haralick_features.ravel())
    
    # Local Binary Pattern (LBP) features
    lbp_radius = 3
    lbp_points = 8 * lbp_radius
    lbp = local_binary_pattern(image, lbp_points, lbp_radius, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, lbp_points + 3), range=(0, lbp_points + 2))
    lbp_features = hist.astype(np.float32)
    
    # Concatenate all features
    features = np.concatenate((stats_features, texture_features, lbp_features))
    all_features.append(features)
    
    
# Save all features as CSV
df_features = pd.DataFrame(all_features)
df_features.to_csv('ExtractedFeatures.csv', index=False)




/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  rem